In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [43]:
df = pd.read_csv("C:\\Users\\91703\\Downloads\\EcoPack.csv")

In [44]:
df.head()

,material_name,strength,weight_capacity_kg,cost_per_unit,biodegradability_score,co2_emission_score,recyclability_percent
0,Corrugated Cardboard,2,12,2,8,3,85
1,Corrugated Cardboard,2,14,2,8,3,88
2,Corrugated Cardboard,3,16,3,7,4,80
3,Kraft Paper,1,5,1,9,2,90
4,Kraft Paper,1,6,1,9,2,92


In [45]:
X = df[['strength',
        'weight_capacity_kg',
        'recyclability_percent',
        'biodegradability_score']]
y_cost = df['cost_per_unit']
y_co2 = df['co2_emission_score']

Splitting Data

In [46]:
X_train, X_test, y_cost_train, y_cost_test = train_test_split(
    X, y_cost, test_size=0.2, random_state=42
)

In [47]:
_, _, y_co2_train, y_co2_test = train_test_split(
    X, y_co2, test_size=0.2, random_state=42
)

In [48]:
print(X_train.shape, X_test.shape)
print(y_cost_train.shape, y_cost_test.shape)
print(y_co2_train.shape, y_co2_test.shape)

(36, 4) (10, 4)
(36,) (10,)
(36,) (10,)


In [49]:
print("Total records:", len(df))
print("Training records:", len(X_train))
print("Testing records:", len(X_test))

Total records: 46
Training records: 36
Testing records: 10


Scaling

In [50]:
from sklearn.preprocessing import StandardScaler

In [51]:
scaler = StandardScaler()

In [52]:
X_train_scaled = scaler.fit_transform(X_train)

In [53]:
X_test_scaled = scaler.transform(X_test)

In [54]:
import numpy as np

print("Mean of scaled training data:", np.mean(X_train_scaled, axis=0))
print("Std of scaled training data:", np.std(X_train_scaled, axis=0))

Mean of scaled training data: [ 2.68303898e-16 -6.78469626e-17  1.00536863e-15 -1.85037171e-16]
Std of scaled training data: [1. 1. 1. 1.]


Random Forest Regressor for cost prediction

In [55]:
from sklearn.ensemble import RandomForestRegressor

In [56]:
rf_cost = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

In [57]:
rf_cost.fit(X_train_scaled, y_cost_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [58]:
y_cost_pred = rf_cost.predict(X_test_scaled)

In [59]:
print("Predicted Cost:", y_cost_pred[:5])
print("Actual Cost:", y_cost_test.values[:5])

Predicted Cost: [3.86 4.   2.94 2.25 3.8 ]
Actual Cost: [4 4 3 2 4]


In [60]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

print("Cost MAE:", mean_absolute_error(y_cost_test, y_cost_pred))
print("Cost R2:", r2_score(y_cost_test, y_cost_pred))
print("Cost RMSE",np.sqrt(mean_squared_error(y_cost_test, y_cost_pred)))

Cost MAE: 0.422
Cost R2: 0.7771323529411766
Cost RMSE 0.5505451843400322


xgboost for co2 prediction

In [61]:
from xgboost import XGBRegressor

In [62]:
xgb_co2 = XGBRegressor(
    n_estimators=100,
    learning_rate=0.2,
    max_depth=4,
    random_state=42
)

In [63]:
xgb_co2.fit(X_train_scaled, y_co2_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes

In [64]:
y_co2_pred = xgb_co2.predict(X_test_scaled)

In [65]:
print("Predicted Cost:", y_co2_pred[:5])
print("Actual Cost:", y_co2_test.values[:5])

Predicted Cost: [6.000607  3.9602573 2.6987329 2.9957254 3.3897932]
Actual Cost: [6 4 3 3 3]


In [66]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

print("CO2 MAE:", mean_absolute_error(y_co2_test, y_co2_pred))
print("CO2 R2:", r2_score(y_co2_test, y_co2_pred))
print("CO2 RMSE",np.sqrt(mean_squared_error(y_co2_test, y_co2_pred)))

CO2 MAE: 0.2360149323940277
CO2 R2: 0.8936170339584351
CO2 RMSE 0.37045109967838136


In [67]:
print(y_cost_train.head())

6     3
24    4
32    4
19    4
17    3
Name: cost_per_unit, dtype: int64


In [68]:
print(X_train_scaled.mean())

2.498001805406602e-16


Normalization

In [69]:
cost_norm = (df['cost_per_unit'] - df['cost_per_unit'].min()) / (
    df['cost_per_unit'].max() - df['cost_per_unit'].min()
)

co2_norm = (df['co2_emission_score'] - df['co2_emission_score'].min()) / (
    df['co2_emission_score'].max() - df['co2_emission_score'].min()
)

In [70]:
# normalize weight capacity to 0–10 scale
df['weight_norm'] = 10 * (df['weight_capacity_kg'] - df['weight_capacity_kg'].min()) / \
                    (df['weight_capacity_kg'].max() - df['weight_capacity_kg'].min())

In [71]:
df['strength_norm'] = (df['strength'] / 3) * 10

df['recyclability_norm'] = df['recyclability_percent'] / 10

In [72]:
df['strength_score'] = (df['strength'] + df['weight_norm']) / 2

In [73]:
df['eco_score'] = (
    df['biodegradability_score'] +
    df['recyclability_percent']/10 -
    df['co2_emission_score']
) / 3

In [74]:
df['suitability_score'] = (
    df['strength_norm'] +
    df['recyclability_norm'] +
    df['biodegradability_score']
) / 3

In [75]:
df['final_score'] = (
    0.4 * cost_norm +
    0.4 * co2_norm +
    0.2 * df['suitability_score']
)

ranked_materials = df.sort_values('final_score',ascending=False)

Raning Materials

In [76]:
ranked_materials[['material_name', 'final_score']].head(10) 

,material_name,final_score
34,Hemp Fiber,2.393333
33,Hemp Fiber,2.373333
9,Bamboo Fiber,2.360000
23,Mushroom Packaging,2.360000
31,Jute Fiber,2.360000
10,Bamboo Fiber,2.346667
16,Honeycomb Paper,2.340000
30,Jute Fiber,2.340000
15,Honeycomb Paper,2.326667
42,Agro-Waste Composite,2.306667


Saving the model

In [77]:
import joblib
import os

In [78]:
os.makedirs("models", exist_ok=True)

In [79]:
joblib.dump(rf_cost, "models/rf_cost_model.pkl")

['models/rf_cost_model.pkl']

In [80]:
joblib.dump(xgb_co2, "models/xgb_co2_model.pkl")

['models/xgb_co2_model.pkl']

In [81]:
joblib.dump(scaler, "models/scaler.pkl")

['models/scaler.pkl']